## PARALELISMO E CLUSTER

### TRABALHAR COM CORES
Para adicionar cores utilize:
```julia
addproces(n)
```
Sendo **n** o número de cores que deseja criar as instâncias. Uma boa dica é usar ``addroces()`` dessa forma Julia cria automaticamente o número de cores que melhor se ajusta à máquina.

In [5]:
using Distributed

In [39]:
# visualizar o numero de cores do processador físico
Sys.cpu_summary()

Intel(R) Core(TM) i7-7700HQ CPU @ 2.80GHz: 
       speed         user         nice          sys         idle          irq
#1  3346 MHz      13220 s         16 s       3009 s      77949 s          0 s
#2  2622 MHz      12725 s         15 s       2394 s      58512 s          0 s
#3  2903 MHz      11831 s         24 s       2266 s      59926 s          0 s
#4  3304 MHz      12284 s         18 s       2254 s      59405 s          0 s
#5  3372 MHz      11560 s         15 s       2060 s      59812 s          0 s
#6  3310 MHz      11997 s         17 s       2044 s      59591 s          0 s
#7  2336 MHz      12966 s         21 s       2043 s      58567 s          0 s
#8  1997 MHz      11750 s         12 s       2172 s      59569 s          0 s


In [7]:
# verificar o número de cores alocados e ID
procs()

1-element Array{Int64,1}:
 1

In [9]:
# adicionar o numero de cores do computador
addprocs()

8-element Array{Int64,1}:
  4
  5
  6
  7
  8
  9
 10
 11

In [10]:
# remover cores rmprocs(ID_CORE)
# para remover mais de um core use rmprocess([vetor_IDCORE])
# Para remover todos os alocados use rmprocs(procs()) 
rmprocs(procs())

┌ Warning: rmprocs: process 1 not removed
└ @ Distributed /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.0/Distributed/src/cluster.jl:932


Task (done) @0x00007f7571068a90

In [11]:
procs()

1-element Array{Int64,1}:
 1

### PARALLEL FOR E  PARALLEL MAP

Um loop for com um grande número de iterações e compreensões de lista, são melhores desempenhados com execução paralela através  da macro `@parallel`. Sintaxe:
```julia
@parallel (operador) for var = range
    instruções
end
```

In [40]:
using BenchmarkTools

**Função Somatório**

In [41]:
# Função Soma sem paralelismo
# Laço for de uma soma de 1 a 1000000000
# de números aleatórios

function serial_add(n::Int64)
    s = 0.0
    for i = 1:n
        s = s + randn()
    end
    return s
end 

serial_add (generic function with 1 method)

In [42]:
# execute mais de uma vez
@benchmark serial_add(1000000000)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     4.134 s (0.00% GC)
  median time:      4.212 s (0.00% GC)
  mean time:        4.212 s (0.00% GC)
  maximum time:     4.291 s (0.00% GC)
  --------------
  samples:          2
  evals/sample:     1

Preparando o paralelismo

In [45]:
# verificar o número de processadores
procs()

1-element Array{Int64,1}:
 1

In [46]:
addprocs()

8-element Array{Int64,1}:
 10
 11
 12
 13
 14
 15
 16
 17

In [47]:
# função paralela

function paralelo_add(n::Int64)
    
  @distributed (+) for i = 1:n
        randn()                  
    end
    
end

paralelo_add (generic function with 1 method)

In [48]:
# execute mais de uma vez
@benchmark paralelo_add(1000000000)

BenchmarkTools.Trial: 
  memory estimate:  67.81 KiB
  allocs estimate:  816
  --------------
  minimum time:     912.697 ms (0.00% GC)
  median time:      940.306 ms (0.00% GC)
  mean time:        938.593 ms (0.00% GC)
  maximum time:     957.156 ms (0.00% GC)
  --------------
  samples:          6
  evals/sample:     1

**Função alocação de dados em uma Matriz**

In [27]:
# Função sem paralelismo

function alo_matriz2(n::Int64)
    matriz = zeros(n, n)
    for i = 1:n
        for j = 1:n
            matriz[i, j] = rand()*0.25
        end
    end
    return matriz
end

alo_matriz2 (generic function with 1 method)

In [25]:
@benchmark alo_matriz2(10000)

BenchmarkTools.Trial: 
  memory estimate:  762.94 MiB
  allocs estimate:  2
  --------------
  minimum time:     1.090 s (2.54% GC)
  median time:      1.120 s (4.42% GC)
  mean time:        1.128 s (3.70% GC)
  maximum time:     1.195 s (4.70% GC)
  --------------
  samples:          5
  evals/sample:     1

In [52]:
GC.gc()

Worker 11 terminated.
Worker 10 terminated.
Worker 13 terminated.
Worker 16 terminated.
Worker 15 terminated.
Worker 14 terminated.
Worker 12 terminated.


In [49]:
# Função paralela

function alo_matriz1(n::Int64)
    matriz = zeros(n, n)
    @distributed for i = 1:n
                for j = 1:n
                    matriz[i, j] = rand()*0.25
                end
              end
    return matriz
end

alo_matriz1 (generic function with 1 method)

In [50]:
@benchmark alo_matriz1(10000)

OutOfMemoryError: OutOfMemoryError()

In [40]:
@elapsed alo_matriz1(10000)

0.223852437

In [44]:
# use o garbage collector para remover o lixo da memória
GC.gc()
GC.gc()

### ParallelAccelerator

ParallelAccelerator é um pacote para acelerar programas de Julia voltados principalmente para o cálculo de matrizes e vetores. Através da macro `@acc` **ParallelAccelerator** compila partes do programa e elimina automaticamente funções desnecessárias e também paraleliza e vetoriza muitas operações de dados em paralelo. Além de acelerar o cálculo, `@acc` optimiza o consumo de memória, logo é muito útil para grandes matrizes. 

ParallelAccelerator é parte do projeto de script de alto desempenho (HPS) da Intel Labs. 



In [51]:
Pkg.add("ParallelAccelerator")

 Resolving package versions...


Pkg.Types.ResolverError: Unsatisfiable requirements detected for package CompilerTools [98f049d2]:
 CompilerTools [98f049d2] log:
 ├─possible versions are: [0.1.0-0.1.7, 0.2.0-0.2.1, 0.3.0-0.3.1] or uninstalled
 ├─restricted by compatibility requirements with ParallelAccelerator [ee84a2f6] to versions: [0.1.0-0.1.7, 0.2.0-0.2.1, 0.3.0-0.3.1]
 │ └─ParallelAccelerator [ee84a2f6] log:
 │   ├─possible versions are: [0.1.0-0.1.9, 0.2.0-0.2.2, 0.3.2] or uninstalled
 │   └─restricted to versions * by an explicit requirement, leaving only versions [0.1.0-0.1.9, 0.2.0-0.2.2, 0.3.2]
 └─restricted by julia compatibility requirements to versions: uninstalled — no versions left

In [ ]:
using BenchmarkTools

In [23]:
NP(n) = [i^2 + j^3 + i^5 + j^7 + i^9 + j^11 for i = 1:n, j = 1:n];

In [24]:
@benchmark NP(10000)

BenchmarkTools.Trial: 
  samples:          2
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%
  memory estimate:  762.94 mb
  allocs estimate:  3
  minimum time:     3.91 s (0.32% GC)
  median time:      3.93 s (1.31% GC)
  mean time:        3.93 s (1.31% GC)
  maximum time:     3.96 s (2.29% GC)

In [25]:
# use o garbage collector para remover o lixo da memória
gc()
gc()

In [26]:
# verificar os cores alocados. Se houver mais de um até o número de cores físicos OK.
procs()

5-element Array{Int64,1}:
  1
 10
 11
 12
 13

In [14]:
# se houver apenas um cores execute
addprocs()

4-element Array{Int64,1}:
 6
 7
 8
 9

In [104]:
Pkg.add("ParallelAccelerator")

INFO: Nothing to be done
INFO: METADATA is out-of-date — you may not have the latest version of ParallelAccelerator
INFO: Use `Pkg.update()` to get the latest versions of your packages


In [29]:
using ParallelAccelerator

ArgumentError: ArgumentError: Package ParallelAccelerator not found in current path:
- Run `import Pkg; Pkg.add("ParallelAccelerator")` to install the ParallelAccelerator package.


In [30]:
@acc PA(n) = [i.^2 + j.^3 + i.^5 + j.^7 + i.^9 + j^.11 for i = 1:n, j = 1:n];

LoadError: UndefVarError: @acc not defined

In [29]:
# execute sempre mais de uma vez para obter o melhor resultado
@timev PA(10000);

 35.374035 seconds (26.60 M allocations: 1.831 GB, 1.86% gc time)
elapsed time (ns): 35374034717
gc time (ns):      656413004
bytes allocated:   1965542385
pool allocs:       26599117
non-pool GC allocs:1370
malloc() calls:    47
GC pauses:         51
full collections:  3


In [30]:
# use o garbage collector para remover o lixo da memória
gc()
gc()

### DistributedArrays

In [1]:
using BenchmarkTools

In [2]:
function array(n)
    return ([i^2 + j^3 + i^5 + j^7 + i^9 + j^11 for i = 1:n, j = 1:n]);
end

array (generic function with 1 method)

In [18]:
@benchmark array(7000)

BenchmarkTools.Trial: 
  memory estimate:  373.84 MiB
  allocs estimate:  2
  --------------
  minimum time:     891.679 ms (1.04% GC)
  median time:      906.971 ms (3.34% GC)
  mean time:        911.770 ms (3.17% GC)
  maximum time:     938.516 ms (4.96% GC)
  --------------
  samples:          6
  evals/sample:     1

In [19]:
# use o garbage collector para remover o lixo da memória
GC.gc()
GC.gc()

![Execução Sem Paralelismo](Figuras/Arrays-sem-paralelismo.png)

Antes de executar o código é necesário verificar se há outros cores disponíveis. Caso exista somente 1, é necessário adicionar novos cores e vincular o `DistributedArrays` ao macro `@everywhere` seguindo a ordem:

    1º adicione os cores addprocs(n*)
    2º use @everywhere using DistributedArrays
    3º execute o código com o macro @Darray 
**n** corresponde ao número de cores. Utilizando `addprocs()`(o mesmo que usar ``addprocs(Sys.CPU_CORES)``) Julia cria automaticamente o número de cores que melhor se ajustam à máquina. Sempre execute uma segunda vez o código, a primeira tem péssimo desempenho.

Caso execute novamente `addprocs(n)` e depois `@Darray`, os cores novos serão eliminados da lista de processos e será utilizado somente os cores vinculados ao macro `@everywhere`. 

In [7]:
using Distributed

In [8]:
# visualizar o numero de cores do processador físico
Sys.CPU_CORES

UndefVarError: UndefVarError: CPU_CORES not defined

In [9]:
# verificar o número de cores disponíveis e ID
procs()

1-element Array{Int64,1}:
 1

In [10]:
# adiciona automaticanente a melhor quantidade de acordo com a quantidade de cores disponíveisa
addprocs()

8-element Array{Int64,1}:
 2
 3
 4
 5
 6
 7
 8
 9

In [11]:
# vincular aos cores
@everywhere using DistributedArrays

In [20]:
function Darray(n)
    return @DArray ([i^2 + j^3 + i^5 + j^7 + i^9 + j^11 for i = 1:n, j = 1:n]);
end

Darray (generic function with 1 method)

In [21]:
@benchmark  Darray(7000)

BenchmarkTools.Trial: 
  memory estimate:  95.61 KiB
  allocs estimate:  1054
  --------------
  minimum time:     216.889 ms (0.00% GC)
  median time:      276.754 ms (0.00% GC)
  mean time:        271.786 ms (0.00% GC)
  maximum time:     365.115 ms (0.00% GC)
  --------------
  samples:          19
  evals/sample:     1

![](Figuras/Arrays-com-paralelismo.png)

In [22]:
# use o garbage collector para remover o lixo da memória
GC.gc()
GC.gc()

In [23]:
#remover os nucleos alocados 
rmprocs(procs())

┌ Warning: rmprocs: process 1 not removed
└ @ Distributed /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.0/Distributed/src/cluster.jl:932


Task (done) @0x00007f3201443730

## CLUSTER